# Clases de secciones

In [2]:
%matplotlib inline

In [3]:
import math

class Seccion:
  def __init__(self,dimensiones:tuple[float|int,float|int|None,float|int|None]):
    if not isinstance(dimensiones,tuple):
      raise TypeError("Las dimensiones deben ser una tupla")

    if "CircularMacizo" in self.__class__.__name__ :
      assert len(dimensiones) == 1, "(radio)"
      self.r = dimensiones
      self.x, self.y, self.e = 0, 0, 0
      self.descripcion = f"(radio = {self.r} mm)"

    elif "CircularHueco" in self.__class__.__name__ :
      assert len(dimensiones) == 2, "(radio, espesor)"
      self.r, self.e = dimensiones
      self.x, self.y = None, None
      self.descripcion = f"(radio = {self.r} mm, espesor = {self.e} mm)"

    elif "RectangularMacizo" in self.__class__.__name__ :
      assert len(dimensiones) == 2, "(longitud, ancho)"
      self.x, self.y =  dimensiones
      self.r, self.e = None, None
      self.descripcion = f"(longitud = {self.x} mm, ancho = {self.y} mm)"

    elif "RectangularHueco" in self.__class__.__name__ :
      assert len(dimensiones) == 3, "(longitud, ancho, espesor)"
      self.x, self.y, self.e =  dimensiones
      self.r = None
      self.descripcion = f"(longitud = {self.x} mm, ancho = {self.y} mm, espesor = {self.e} mm)"

    elif "CuadradoHueco" in self.__class__.__name__ :
      assert len(dimensiones) == 2, "(lado, espesor)"
      self.x, self.e =  dimensiones
      self.y = self.x
      self.r = None
      self.descripcion = f"(lado = {self.x} mm, espesor = {self.e} mm)"

    elif "CuadradoMacizo" in self.__class__.__name__ :
      assert len(dimensiones) == 1, "(lado)"
      self.x =  dimensiones
      self.y = self.x
      self.r, self.e = None, None
      self.descripcion = f"(lado = {self.x} mm)"

  @property
  def tipo(self):
    return f"{self.__class__.__name__}"

  def __repr__(self):
    return f'''---- {self.__class__.__name__} ----
    \nReferencia (0,0) desde la esquina superior izquierda
    \n---- Sección ----\nx (longitud) = {self.x} mm\ny (ancho) = {self.y} mm\ne (espesor) = {self.e} mm\nr (radio) = {self.r} mm
    \n---- Área y centroide ----\nArea: {self.area_centroide[0]:.1f} mm2\nPosición centroide (x,y): {self.area_centroide[1]}
    \n---- Inercias ----\nIx: {self.momentos_inercia[0]/1e4:,.1f} cm4\nIy: {self.momentos_inercia[1]/1e4:,.1f} cm4
    \n---- Modulos resistentes ----\nWx: {self.modulos_resistentes[0]/1e3:,.1f} cm3\nWy: {self.modulos_resistentes[1]/1e3:,.1f} cm3
    \n------------
     '''

class SeccionRectangularMacizo(Seccion):
  def __init__(self,dimensiones:tuple[float|int,float|int|None,float|int|None]):
    ''' El eje x es el eje horizontal,\n
    el eje y es el eje vertical.\n
    Para las dimensiones escribir siempre (x,y) en este orden.'''
    super().__init__(dimensiones)

  @property
  def area_centroide(self):
    ''' Devuelve el area y la posición del centroide respecto a la
    esquina superior izquierda. Siempre en mm.'''
    area = self.x * self.y
    self.centroide = (self.x / 2, self.y / 2)
    return area, self.centroide

  @property
  def momentos_inercia(self):
    ''' Devuelve los momentos de inercia de la seccion con respecto
    al centroide de la sección. en mm4 '''
    Ix = (self.x * self.y ** 3) / 12
    Iy = (self.y * self.x ** 3) / 12
    return Ix, Iy

  @property
  def modulos_resistentes(self):
    ''' Devuelve los modulos resistentes Wx, Wy de la seccion
    con respecto al centroide '''
    Ix, Iy = self.momentos_inercia
    x, y = self.centroide
    Wx = Ix / x
    Wy = Iy / y
    return Wx, Wy

class SeccionRectangularHueco(Seccion):
  def __init__(self,dimensiones:tuple[float|int,float|int|None,float|int|None]):
    ''' El eje x es el eje horizontal,\n
    el eje y es el eje vertical.\n
    e es el espesor.
    (longitud,ancho,espesor)
    Para las dimensiones escribir siempre (x,y,e) en este orden.'''
    super().__init__(dimensiones)

  @property
  def area_centroide(self):
    ''' Devuelve el area y la posición del centroide respecto a la
    esquina superior izquierda. Siempre en mm.'''
    area_total = self.x * self.y
    area_hueco = (self.x - 2 * self.e) * (self.y - 2 * self.e)
    area = area_total - area_hueco
    self.centroide = (self.x / 2, self.y / 2)
    return area, self.centroide

  @property
  def momentos_inercia(self):
    ''' Devuelve los momentos de inercia de la seccion con respecto
    al centroide de la sección. en mm4 '''
    Ix_total = (self.x * self.y ** 3) / 12
    Iy_total = (self.y * self.x ** 3) / 12
    longitud_hueco = self.x - 2 * self.e
    ancho_hueco = self.y - 2 * self.e
    Ix_hueco = (longitud_hueco * ancho_hueco ** 3) / 12
    Iy_hueco = (ancho_hueco * longitud_hueco ** 3) / 12
    Ix = Ix_total - Ix_hueco
    Iy = Iy_total - Iy_hueco
    return Ix, Iy

  @property
  def modulos_resistentes(self):
    ''' Devuelve los modulos resistentes Wx, Wy de la seccion
    con respecto al centroide '''
    Ix, Iy = self.momentos_inercia
    x, y = self.centroide
    Wx = Ix / y
    Wy = Iy / x
    return Wx, Wy

class SeccionCircularHueco(Seccion):
  ''' Introducir el radio de la sección. '''
  def __init__(self,dimensiones:tuple[float|int,float|int|None,float|int|None]):
    ''' El eje x es el eje horizontal,\n
    el eje y es el eje vertical.\n
    e es el espesor.
    (longitud,ancho,espesor)
    Para las dimensiones escribir siempre (x,y,e) en este orden.'''
    super().__init__(dimensiones)

  @property
  def area_centroide(self):
    ''' Devuelve el area y la posición del centroide respecto al
    centro de la circunferencia. Siempre en mm.'''
    radio_interior = self.r - self.e
    area_total = math.pi * self.r ** 2
    area_hueco = math.pi * radio_interior ** 2
    area = area_total - area_hueco
    self.centroide = (0, 0)
    return area, self.centroide

  @property
  def momentos_inercia(self):
    ''' Devuelve los momentos de inercia de la seccion con respecto
    al centroide de la sección. '''
    radio_hueco = self.r - self.e
    Ix_total = Iy_total = (math.pi * self.r ** 4) / 4
    Ix_hueco = Iy_hueco = (math.pi * radio_hueco ** 4) / 4
    Ix = Ix_total - Ix_hueco
    Iy = Iy_total - Iy_hueco
    return Ix, Iy

  @property
  def modulos_resistentes(self):
    ''' Devuelve los modulos resistentes Wx, Wy de la seccion
    con respecto al centroide '''
    Ix, Iy = self.momentos_inercia
    x, y = self.centroide
    Wx = Wy = Ix / self.r
    return Wx, Wy

class SeccionCircularMacizo(Seccion):
  ''' Introducir el radio de la sección. '''
  def __init__(self,dimensiones:tuple[float|int,float|int|None,float|int|None]):
    ''' El eje x es el eje horizontal,\n
    el eje y es el eje vertical.\n
    e es el espesor.
    (longitud,ancho,espesor)
    Para las dimensiones escribir siempre (x,y,e) en este orden.'''
    super().__init__(dimensiones)

  @property
  def area_centroide(self):
    ''' Devuelve el area y la posición del centroide respecto al
    centro de la circunferencia. Siempre en mm.'''
    area = math.pi * self.r ** 2
    self.centroide = (0, 0)
    return area, self.centroide

  @property
  def momentos_inercia(self):
    ''' Devuelve los momentos de inercia de la seccion con respecto
    al centroide de la sección. '''
    Ix = Iy = (math.pi * self.r ** 4) / 4
    return Ix, Iy

  @property
  def modulos_resistentes(self):
    ''' Devuelve los modulos resistentes Wx, Wy de la seccion
    con respecto al centroide '''
    Ix, Iy = self.momentos_inercia
    x, y = self.centroide
    Wx = Wy = Ix / self.r
    return Wx, Wy

class SeccionCuadradoHueco(Seccion):
  def __init__(self,dimensiones:tuple[float|int,float|int|None,float|int|None]):
    ''' El eje x es el eje horizontal,\n
    el eje y es el eje vertical.\n
    e es el espesor.
    (longitud,ancho,espesor)
    Para las dimensiones escribir siempre (x,y,e) en este orden.'''
    super().__init__(dimensiones)

  @property
  def area_centroide(self):
    ''' Devuelve el area y la posición del centroide respecto a la
    esquina superior izquierda. Siempre en mm.'''
    area_total = self.x ** 2
    area_hueco = (self.x - 2 * self.e) ** 2
    area = area_total - area_hueco
    self.centroide = (self.x / 2, self.x / 2)
    return area, self.centroide

  @property
  def momentos_inercia(self):
    ''' Devuelve los momentos de inercia de la seccion con respecto
    al centroide de la sección. '''
    lado_hueco = self.x - 2 * self.e
    Ix_total = Iy_total = (self.x ** 4) / 12
    Ix_hueco = Iy_hueco = (lado_hueco ** 4) / 12
    Ix = Ix_total - Ix_hueco
    Iy = Iy_total - Iy_hueco
    return Ix, Iy

  @property
  def modulos_resistentes(self):
    ''' Devuelve los modulos resistentes Wx, Wy de la seccion
    con respecto al centroide '''
    Ix, Iy = self.momentos_inercia
    x, y = self.centroide
    Wx = Ix / y
    Wy = Iy / x
    return Wx, Wy

class SeccionCuadradoMacizo(Seccion):
  def __init__(self,dimensiones:tuple[float|int,float|int|None,float|int|None]):
    ''' El eje x es el eje horizontal,\n
    el eje y es el eje vertical.\n
    e es el espesor.
    (longitud,ancho,espesor)
    Para las dimensiones escribir siempre (x,y,e) en este orden.'''
    super().__init__(dimensiones)

  @property
  def area_centroide(self):
    ''' Devuelve el area y la posición del centroide respecto a la
    esquina superior izquierda. Siempre en mm.'''
    area = self.x ** 2
    self.centroide = (self.x / 2, self.x / 2)
    return area, self.centroide

  @property
  def momentos_inercia(self):
    ''' Devuelve los momentos de inercia de la seccion con respecto
    al centroide de la sección. '''
    Ix = Iy = (self.x ** 4) / 12
    return Ix, Iy

  @property
  def modulos_resistentes(self):
    ''' Devuelve los modulos resistentes Wx, Wy de la seccion
    con respecto al centroide '''
    Ix, Iy = self.momentos_inercia
    x, y = self.centroide
    Wx = Ix / y
    Wy = Iy / x
    return Wx, Wy

class SeccionCompuesta:
    def __init__(self, secciones:list[dict]):
        self.secciones = secciones

    @property
    def area_centroide(self):
        ''' Devuelve la posición (x,y) del centroide de la sección compuesta en mm.
        La referencia es la esquina superior izquierda más a la izquierda. '''

        area_total = 0
        centroide_total_x = 0
        centroide_total_y = 0

        for seccion in self.secciones:
            ubicacion = seccion['ubicacion']

            area, centroide = seccion["seccion"].area_centroide

            area_total += area
            centroide_total_x += (centroide[0] + ubicacion[0]) * area
            centroide_total_y += (centroide[1] + ubicacion[1]) * area

        centroide_composicion = (centroide_total_x / area_total, centroide_total_y / area_total)

        return area_total,centroide_composicion

    @property
    def momentos_inercia(self):
      _,centroid_composicion = self.area_centroide

      Ix_total = 0
      Iy_total = 0

      for seccion in self.secciones:
          ubicacion = seccion['ubicacion']

          area, centroide_seccion = seccion["seccion"].area_centroide
          Ix, Iy = seccion["seccion"].momentos_inercia

          # Aplicar el teorema del eje paralelo
          dx = abs(centroid_composicion[0] - (ubicacion[0]+centroide_seccion[0]))
          dy = abs(centroid_composicion[1] - (ubicacion[1]+centroide_seccion[1]))

          Ix += area * dy ** 2
          Iy += area * dx ** 2

          Ix_total += Ix
          Iy_total += Iy

      return Ix_total, Iy_total

    @property
    def modulos_resistentes(self):
      #calculamos los puntos mas alejados del centroide
      #Para la inercia Ix, calculamos los puntos y
      _,(x_centroide, y_centroide) = self.area_centroide
      puntos_alejados_x = []
      puntos_alejados_y = []

      for seccion in self.secciones:
        puntos_alejados_x.append(y_centroide - seccion["ubicacion"][1])
        puntos_alejados_y.append(x_centroide - seccion["ubicacion"][0])

      Ix, Iy = self.momentos_inercia
      Wx = Ix / max(puntos_alejados_y)
      Wy = Iy / max(puntos_alejados_x)
      return Wx, Wy

    def __repr__(self):
      informacion_secciones = "SeccionCompuesta(\n"
      for idx,seccion in enumerate(self.secciones,start=1):
        informacion_secciones += f"\t{idx}-" + seccion["seccion"].tipo + seccion["seccion"].descripcion + "\n"
      informacion_secciones + ")\n"

      informacion_secciones += f'''\n---- Propiedades mecánicas ----
      \nReferencia (0,0) desde la esquina superior izquierda
      \n---- Área y centroide ----\nArea Total: {self.area_centroide[0]:.1f} mm2\nPosición centroide (x,y): {self.area_centroide[1]}
      \n---- Inercias ----\nIx: {self.momentos_inercia[0]/1e4:,.1f} cm4\nIy: {self.momentos_inercia[1]/1e4:,.1f} cm4
      \n---- Modulos resistentes ----\nWx: {self.modulos_resistentes[0]/1e3:,.1f} cm3\nWy: {self.modulos_resistentes[1]/1e3:,.1f} cm3
      \n------------
      '''
      return informacion_secciones

In [4]:
pieds_poteau = SeccionCompuesta([
    {
        "seccion" : SeccionRectangularHueco(
            (120,60,2)
        ),
        "ubicacion" : (0,0)
    },
    {
        "seccion" : SeccionCuadradoHueco(
            (55,3)
        ),
        "ubicacion" : (5,2.5)
    },
    {
        "seccion" : SeccionCuadradoHueco(
            (55,3)
        ),
        "ubicacion" : (60,2.5)
    }
])

pieds_poteau

SeccionCompuesta(
	1-SeccionRectangularHueco(longitud = 120 mm, ancho = 60 mm, espesor = 2 mm)
	2-SeccionCuadradoHueco(lado = 55 mm, espesor = 3 mm)
	3-SeccionCuadradoHueco(lado = 55 mm, espesor = 3 mm)

---- Propiedades mecánicas ----
      
Referencia (0,0) desde la esquina superior izquierda
      
---- Área y centroide ----
Area Total: 1952.0 mm2
Posición centroide (x,y): (60.0, 30.0)
      
---- Inercias ----
Ix: 102.7 cm4
Iy: 286.4 cm4
      
---- Modulos resistentes ----
Wx: 17.1 cm3
Wy: 95.5 cm3
      
------------
      

# Visualización de secciones

In [7]:
import cv2 as cv
import numpy as np
from typing import Iterable
import matplotlib.pyplot as plt

class GraficarSeccion:

    def __init__(
            self,
            seccion:Seccion|SeccionCompuesta,
            pad_ventana:tuple[int,int]=(50,50),
            factor_escala:int=5,
            ):
        self.seccion = seccion
        self.PAD_VENTANA = pad_ventana
        self.COLOR_VENTANA = (0,0,0)
        self.COLORES_SECCIONES = [(113, 204, 46),(193, 134, 46),(0, 195, 255),(60, 76, 231)]
        self.ESC = factor_escala
        self.COLOR_BLACK = (0,0,0)
        self.COLOR_WHITE = (255,255,255)
        self.COLOR_RED = (15,15,136)
        self.FONT = cv.FONT_HERSHEY_COMPLEX_SMALL

    def _get_color(self):
        for color in self.COLORES_SECCIONES:
            yield color

    def _definir_dimensiones_ventana(self):
        vent_x = max([ seccion["seccion"].x for seccion in self.seccion.secciones if "Circular" not in seccion["seccion"].tipo])
        vent_y = max([ seccion["seccion"].y for seccion in self.seccion.secciones if "Circular" not in seccion["seccion"].tipo])
        vent_x += 2 * self.PAD_VENTANA[0]
        vent_y += 2 * self.PAD_VENTANA[1]
        return self._escalar((vent_y, vent_x))

    def _escalar(self,valores:Iterable|int)->list:
        #escala y transforma en int para poder dibujarlo
        if not isinstance(valores,Iterable):
            return int(valores * self.ESC)
        else:
            output = []
            for valor in valores:
                output.append(int(self.ESC * valor))
            return output

    def _dibujar_centroide(self,img:np.ndarray,coord:tuple[int,int])->None:
        #Añadimos el padding
        x = coord[0] + self.PAD_VENTANA[0]
        y = coord[1] + self.PAD_VENTANA[1]
        #Escalamos y pasamos a int
        x, y = self._escalar((x,y))
        r = self._escalar(3)
        cv.circle(img,center=(x,y),radius=r,color=self.COLOR_WHITE,thickness=-1)
        cv.ellipse(img,(x, y), (r, r), 0, 90, 180, self.COLOR_RED, cv.FILLED)
        cv.ellipse(img,(x, y), (r, r), 0, 270, 360, self.COLOR_RED, cv.FILLED)
        long_ejes = self._escalar(10)
        Ix, Iy = self.seccion.momentos_inercia
        Ix, Iy = Ix/1e4, Iy/1e4
        #Eje x
        cv.arrowedLine(img,pt1=(x,y), pt2=(x+long_ejes,y),color=self.COLOR_RED,thickness=2)
        cv.putText(img,f'Ix = {Ix:.1f} cm4',(x+long_ejes,y + self._escalar(-3)), self.FONT, self._escalar(0.25),self.COLOR_RED,1,cv.LINE_AA)
        #Eje y
        cv.arrowedLine(img,pt1=(x,y), pt2=(x,y+long_ejes),color=self.COLOR_RED,thickness=2)
        cv.putText(img,f'Iy = {Iy:.1f} cm4',(x,y+long_ejes+ self._escalar(5)), self.FONT, self._escalar(0.25),self.COLOR_RED,1,cv.LINE_AA)

    def _dibujar_ejes_coord(self,img:np.ndarray)->None:
        ''' Dibuja los ejes de coordenadas x e y de referencia de la figura completa '''
        long_ejes = self._escalar(10)
        x, y = self._escalar(self.PAD_VENTANA)
        #Eje x
        cv.arrowedLine(img,pt1=(x,y), pt2=(x+long_ejes,y),color=self.COLOR_WHITE,thickness=3)
        cv.putText(img,'x',(x+long_ejes,y+self._escalar(-3)), self.FONT, self._escalar(0.25),self.COLOR_WHITE,1,cv.LINE_AA)
        #Eje y
        cv.arrowedLine(img,pt1=(x,y), pt2=(x,y+long_ejes),color=self.COLOR_WHITE,thickness=3)
        cv.putText(img,'y',(x - self._escalar(5),y+long_ejes + self._escalar(5)), self.FONT, self._escalar(0.25),self.COLOR_WHITE,1,cv.LINE_AA)

    def dibujar_seccion(self):
        img = np.zeros((*(self._definir_dimensiones_ventana()),3),dtype=np.uint8)
        color_gen = self._get_color()
        for seccion in self.seccion.secciones:
            #ubicacion de la seccion
            x = seccion["ubicacion"][0]
            y = seccion["ubicacion"][1]
            tipo_seccion = seccion["seccion"].tipo
            #añadimos el padding
            x += self.PAD_VENTANA[0]
            y += self.PAD_VENTANA[1]

            if "Rectangular" in tipo_seccion or "Cuadrado" in tipo_seccion:
                dx, dy = seccion["seccion"].x, seccion["seccion"].y
                x, y, dx, dy = self._escalar((x,y,dx,dy))
                cv.rectangle(img= img,pt1= (x,y),pt2= (x+dx,y+dy), color=next(color_gen),thickness=-1)
            if "Hueco" in tipo_seccion:
                e = self._escalar(seccion["seccion"].e)
                cv.rectangle(img=img, pt1=(x+e,y+e), pt2=(x+dx-e,y+dy-e),color=self.COLOR_BLACK,thickness=-1)
        #Dibujamos el centroide de la seccion compuesta
        self._dibujar_centroide(img,self.seccion.area_centroide[1])
        self._dibujar_ejes_coord(img)
        #TODO Esta parte cambiará en streamlit
        cv.imshow("Secciones",img)
        cv.waitKey(0)
        cv.destroyAllWindows()

In [6]:
grafica_pieds_poteau = GraficarSeccion(pieds_poteau)
grafica_pieds_poteau.dibujar_seccion()

In [8]:
grafica_manchon55x3 = GraficarSeccion(
    SeccionCompuesta([
        {
            "seccion" : SeccionCuadradoHueco(
                (60,1.5)
            ),
            "ubicacion" : (0,0)
        },
        {
            "seccion" : SeccionCuadradoHueco(
                (55,3)
            ),
            "ubicacion" : (2,2)
        },

    ])
)
grafica_manchon55x3.dibujar_seccion()

In [9]:
SECTION_MAPPING = {
    "Tube Carré" : "sc.SeccionCuadradoHueco",
    "Carré plein" : "sc.SeccionCuadradoMacizo",
    "Tube Rectangle" : "sc.SeccionRectangularHueco",
    "Rectangle plein" : "sc.SeccionRectangularMacizo",
    "Tube Rond" : "sc.SeccionCircularHueco",
    "Rond Plein" : "sc.SeccionCircularMacizo",
}

In [18]:
def get_section_names(section_mapping:dict)->list[str]:
    """Devuelve una lista de las secciones para el selectbox

    Parameters
    ----------
    section_mapping : dict
        _description_

    Returns
    -------
    list[str]
        _description_
    """
    print(section_mapping.keys())
    return [sec for sec in section_mapping.keys()]

In [19]:
get_section_names(SECTION_MAPPING)

dict_keys(['Tube Carré', 'Carré plein', 'Tube Rectangle', 'Rectangle plein', 'Tube Rond', 'Rond Plein'])


['Tube Carré',
 'Carré plein',
 'Tube Rectangle',
 'Rectangle plein',
 'Tube Rond',
 'Rond Plein']

In [23]:
COLORES_SECCIONES = [
            (46, 204, 113),
            (46, 134, 193),
            (255, 195, 0),
            (231, 76, 60),
            ]

def _get_color():
    try:
        for color in COLORES_SECCIONES:
            yield color
    except StopIteration:
        red = np.random.randint(0,250)
        green = np.random.randint(0,250)
        blue = np.random.randint(0,250)
        yield (red, green, blue)

In [24]:
color_gen = _get_color()

In [27]:
for color in color_gen:
    print(color)


In [4]:
import numpy as np
img = np.zeros((800,800,3),dtype=np.uint8)
img.shape

(800, 800, 3)

In [5]:
from passlib.hash import pbkdf2_sha256 as ph

password = "dslknvsd sdfm"
hash_password = ph.hash(password)

hash_password

'$pbkdf2-sha256$29000$01pL6R2DkHJuTclZC0HIGQ$xBsq86Qpvcn9e4VYUl4rI.mmcbl6xz.7SIOlRzEAmjg'

In [6]:
ph.verify("dslknvsd sdfm",hash_password)

True

In [7]:
user = "s jsdkfljsd"
splitear = user.encode("utf-8").split()
len(splitear[0]) > 5

False